In [ ]:
# Add your Google Drive.
# NOTE: In my Google Drive. I have model(llama-2-7b-chat.ggmlv3.q8_0.bin) in my_model directory.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain
!pip install ctransformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install pypdf

In [4]:
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [6]:
DATA_PATH='data/'                   # In this folder we have all pdf.
CHUNK_SIZE=500
CHUNK_OVERLAP=50
DB_FAISS_PATH='embed_vector'        # In this folder we store embedded vector.

# Build vector database
def run_db_build():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE,
                                                   chunk_overlap=CHUNK_OVERLAP)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

    vectorstore = FAISS.from_documents(texts, embeddings)
    vectorstore.save_local(DB_FAISS_PATH)

run_db_build()

In [15]:
MODEL_BIN_PATH='/content/drive/MyDrive/my_model/llama-2-7b-chat.ggmlv3.q8_0.bin'
MODEL_TYPE='llama'
MAX_NEW_TOKENS=256
TEMPERATURE=0.7
VECTOR_COUNT=2
RETURN_SOURCE_DOCUMENTS=False


qa_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [16]:
def build_llm():
    # Local CTransformers model
    llm = CTransformers(model=MODEL_BIN_PATH,
                        model_type=MODEL_TYPE,
                        config={'max_new_tokens': MAX_NEW_TOKENS,
                                'temperature': TEMPERATURE}
                        )

    return llm

def set_qa_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=qa_template,
                            input_variables=['context', 'question'])
    return prompt


def build_retrieval_qa(llm, prompt, vectordb):
    dbqa = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=vectordb.as_retriever(search_kwargs={'k': VECTOR_COUNT}),
                                       return_source_documents=RETURN_SOURCE_DOCUMENTS,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return dbqa


def setup_dbqa():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    vectordb = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = build_llm()
    qa_prompt = set_qa_prompt()
    dbqa = build_retrieval_qa(llm, qa_prompt, vectordb)

    return dbqa

In [17]:
def main():
    query = input("Enter your query: ")
    qa_result = setup_dbqa()
    response = qa_result({'query': query})
    print(response)

In [18]:
main()

Enter your query: who said God is in the details?
{'query': 'who said God is in the details?', 'result': 'Ludwig Mies van der Rohe', 'source_documents': [Document(page_content='the kitchen sink. Most arts echo analogous sentiments. In our search for others whoascribe God’ s home as being in the details, we ﬁnd ourselves in the good company of the19th century French author Gustav Flaubert. The French poet Paul Valery advises us that apoem is never done and bears continual rework, and to stop working on it is abandonment.Such preoccupation with detail is common to all endeavors of excellence. So maybe thereis little new here, but in reading this book you will be', metadata={'source': 'data/test.pdf', 'page': 2}), Document(page_content='already wanted to say here. Small things matter. This is a book about humble concernswhose value is nonetheless far from small.\nGod is in the details , said the architect Ludwig mies van der Rohe. This quote recalls', metadata={'source': 'data/test.pdf', 